In [9]:
pip install opencv-python

     ---------------------------------------- 38.2/38.2 MB 5.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade pip

In [15]:
import tkinter as tk
from tkinter import scrolledtext
import time
import sys
import os
import cv2
import pyocr
import pyocr.builders
import pyautogui  
from PIL import Image, ImageTk,ImageEnhance  
from googletrans import Translator
TESSERACT_PATH = 'C:\Program Files\Tesseract-OCR'
TESSDATA_PATH = 'C:\Program Files\Tesseract-OCR\\tessdata'

os.environ["PATH"] += os.pathsep + TESSERACT_PATH
os.environ["TESSDATA_PREFIX"] = TESSDATA_PATH
tools = pyocr.get_available_tools()
if len(tools) == 0:
	print("No OCR tool found")
	sys.exit(1)
# The tools are returned in the recommended order of usage
tool = tools[0]
print("Will use tool '%s'" % (tool.get_name()))
# Ex: Will use tool 'libtesseract'

langs = tool.get_available_languages()
print("Available languages: %s" % ", ".join(langs))
lang = langs[0]
print("Will use lang '%s'" % (lang))

RESIZE_RETIO = 1 # 縮小倍率の規定

def get_select_langage(): #言語のコードをセット
    global langA #OCR用
    global langB #googletrans用
    language = selected_language.get()
    if language =="English":
        langA="eng" 
        langB="en"  
    elif language =="Korean":
        langA="kor"
        langB="ko"
    elif language =="Chinese":
        langA="chi_tra"
        langB="zh-tw"
    elif language =="japanese":
        langA="jpn"
        langB="ja"
def risize_ratio(cut): #切取った画像の縮小比率を返す
    for i in range(0,20,1): 
        ratio=1+i*0.2
        if (cut.width/ratio>=800) or (cut.height/ratio>=800):
            print("ok")
        else:
            return ratio
    return ratio

def translation(output_text):
    tr = Translator()
    result = tr.translate(output_text, src=langB, dest="ja").text
    add_output(text_box2,result)
    
def SetCanvas(capturedImage):
    if capturedImage==True:
        canvas2.create_image(400, 400, image=image_cut,tag="ss")
    else:
        canvas2.delete(tag="ss")
def screenshot():
    global canvasSub,image,subwindow
    image = pyautogui.screenshot()
    width, height = image.size
    
    image_tk = ImageTk.PhotoImage(image)
    
    subwindow = tk.Toplevel(root)
    subwindow.title("Screen Capture")
    
    canvasSub = tk.Canvas(subwindow, width=1920, height=1080)
    canvasSub.pack()
    canvasSub.create_image(0, 0, anchor=tk.NW, image=image_tk)
    canvasSub.bind("<ButtonPress-1>", start_point_get)
    canvasSub.bind("<Button1-Motion>", rect_drawing)
    canvasSub.bind("<ButtonRelease-1>", release_action)
    
    
    canvasSub.image = image_tk
def btn_click():
    root.withdraw()
    time.sleep(0.5)
    screenshot()
def btn_click2():
    text_box1.delete("1.0", tk.END)
    text_box2.delete("1.0", tk.END)
    capturedImage=False
    SetCanvas(capturedImage)
def btn_click3():
    get_select_langage()
    txt = tool.image_to_string(
        Image.open('TransImage.jpg'),
        lang=langA, # 読み取る言語
        builder=pyocr.builders.TextBuilder(tesseract_layout=6)
    )
    # 読みやすいように整形
    output_text = txt.replace('\n',' ').replace('  ',' ').replace('  ',' ').replace('.','.\n')
    add_output(text_box1,"\n【原文】\n------------------------------------")
    add_output(text_box1,output_text)
    translation(output_text)
    print("------------------------------------")


# ドラッグ開始した時のイベント
def start_point_get(event):
    global start_x, start_y 

    canvasSub.delete("rect1")  

    canvasSub.create_rectangle(event.x,
                             event.y,
                             event.x + 1,
                             event.y + 1,
                             outline="red",
                             tag="rect1")
    # グローバル変数に座標を格納
    start_x, start_y = event.x, event.y

# ドラッグ中のイベント 
def rect_drawing(event):
    #global end_x, end_y
    # マウスポインタが領域外に出た時の処理
    if event.x < 0:
        end_x = 0
    else:
        end_x = min(image.width, event.x)
    if event.y < 0:
        end_y = 0
    else:
        end_y = min(image.height, event.y)

    # "rect1"タグの画像を再描画
    canvasSub.coords("rect1", start_x, start_y, end_x, end_y)

# ドラッグを離したときのイベント 
def release_action(event):
    global image_cut
    start_x, start_y, end_x, end_y = [
        round(n) for n in canvasSub.coords("rect1")
    ]
    subwindow.withdraw()
    root.deiconify()
    print(start_x)
    print(start_y)
    print(end_x)
    print(end_y)
    print(RESIZE_RETIO)
    cut=image.crop((start_x, start_y, end_x, end_y))
    RESIZE=risize_ratio(cut)
    cutre = cut.resize(size=(int(cut.width / RESIZE),
                                 int(cut.height / RESIZE)),
                          resample=Image.BILINEAR)
    image_clear=ClearImage(cutre)
    image_clear.save('TransImage.jpg')
    image_cut = ImageTk.PhotoImage(cutre)
    capturedImage=True
    SetCanvas(capturedImage)
def add_output(box,text):#指定したテキストボックスに、指定した文字を出力する
    box.insert(tk.END, text + "\n") 
def ClearImage(img):#画像を識別しやすくする
    img_g = img.convert('L') 
    enhancer= ImageEnhance.Contrast(img_g) 
    img_con = enhancer.enhance(2.0) 
    return img_con
if __name__ == "__main__":
    global root
    capturedImage=False;
    languages = ["English", "Korean", "Chinese","japanese"] #言語選択
    #メイン画面
    root = tk.Tk()
    root.geometry("1200x800")
    frame_tool_bar = tk.Frame(root, borderwidth = 2,relief = tk.SUNKEN)
    buttonA = tk.Button(frame_tool_bar, text = '切り取り',height=2,foreground='white',background='red',command=btn_click)
    buttonB = tk.Button(frame_tool_bar, text = '取り消す',height=2,foreground='white',background='red',command=btn_click2)
    buttonC = tk.Button(frame_tool_bar, text = '翻訳',height=2,foreground='white',background='red',command=btn_click3)
    label1=tk.Label(frame_tool_bar,text="翻訳する言語")
    buttonA.pack(side = tk.LEFT)
    buttonB.pack(side = tk.LEFT)
    buttonC.pack(side = tk.LEFT)
    selected_language = tk.StringVar()
    selected_language.set(languages[0])
    select_box = tk.OptionMenu(frame_tool_bar, selected_language, *languages)
    select_box.pack(side = tk.RIGHT)
    frame_tool_bar.pack(fill = tk.X)
    label1.pack(side = tk.RIGHT)
    canvas2 = tk.Canvas(root,
                             bg="white",
                             width=800,
                             height=800)
    canvas2.pack(side=tk.LEFT)
    label2=tk.Label(root,text="原文",font=("normal",15))
    label2.pack(pady=(30,0))
    text_box1 = scrolledtext.ScrolledText(root, width=50, height=20)
    text_box1.pack(anchor=tk.NE)
    label3=tk.Label(root,text="翻訳結果",font=("normal",15))
    label3.pack(pady=(30,0))
    text_box2 = scrolledtext.ScrolledText(root, width=50, height=20)
    text_box2.pack(anchor=tk.NE)

    root.mainloop()


Will use tool 'Tesseract (sh)'
Available languages: afr, amh, ara, asm, aze, aze_cyrl, bel, ben, bod, bos, bre, bul, cat, ceb, ces, chi_sim, chi_sim_vert, chi_tra, chi_tra_vert, chr, cos, cym, dan, deu, div, dzo, ell, eng, enm, epo, equ, est, eus, fao, fas, fil, fin, fra, frk, frm, fry, gla, gle, glg, grc, guj, hat, heb, hin, hrv, hun, hye, iku, ind, isl, ita, ita_old, jav, jpn, jpn_vert, kan, kat, kat_old, kaz, khm, kir, kmr, kor, lao, lat, lav, lit, ltz, mal, mar, mkd, mlt, mon, mri, msa, mya, nep, nld, nor, oci, ori, osd, pan, pol, por, pus, que, ron, rus, san, script/Arabic, script/Armenian, script/Bengali, script/Canadian_Aboriginal, script/Cherokee, script/Cyrillic, script/Devanagari, script/Ethiopic, script/Fraktur, script/Georgian, script/Greek, script/Gujarati, script/Gurmukhi, script/HanS, script/HanS_vert, script/HanT, script/HanT_vert, script/Hangul, script/Hangul_vert, script/Hebrew, script/Japanese, script/Japanese_vert, script/Kannada, script/Khmer, script/Lao, script/La